**Tester le code avec des événements plus probable!!!**
# To do list:
* Q1-1
#####* Valeurs théoriques? (Peng-Wei)
#####* Splitting (Diane)
#####* Echantillonage d'importance (Peng-Wei)
#####* Chaîne de Markov -> Regarder l'article FodraPhamMicrostructure2015

* Q1-2
#####* Quantile processus AR(1) (comme dans calcul liste_a Q1-1) (Diane)
#####* Quantile échantillonnage d'importance (Diane)
* Q2
#####* La même chose mais il faut prendre en compte les différents instants de saut. Donc ça change tout.
#####* M-C (Peng-Wei

* Q
* Q3
# Finished:
* Q1
#####* Monte-Carlo naïve


In [1]:
import numpy as np
import math
import matplotlib.pyplot

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [0]:
print("Test : Quelle est la méthode la plus éfficace pour créer des v.a. prenant valeurs dans {-1, 1} de proba 0.5 pour chaque")
print("Binomial")
%timeit -n 5 np.random.binomial(n = 1, p = 0.5, size = int(1e5)) * 2 - 1

print("Uniform")
def propre_binomial(size):
  x = np.random.uniform(size=size)
  x[x <= 0.5] = -1
  x[x > 0.5] = 1
  return x
%timeit -n 5 propre_binomial(int(1e5))

print("Random choice(numpy)")
%timeit -n 5 np.random.choice([-1, 1], size=int(1e5), p=[1/2, 1/2])

Test : Quelle est la méthode la plus éfficace pour créer des v.a. prenant valeurs dans {-1, 1} de proba 0.5 pour chaque
Binomial
5 loops, best of 3: 2.9 ms per loop
Uniform
5 loops, best of 3: 2.54 ms per loop
Random choice(numpy)
5 loops, best of 3: 2.38 ms per loop


In [0]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

saut_1 = lambda x: np.random.choice([-1, 1], size=x, p=[0.5, 0.5])
saut_2 = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6]))
P0 = 35
T = 4*60*60
lamb = 1/300

### Q1 -1 Trajectoire

In [0]:
## Version Diane Naif
%%time

P0 = 15 #35 en théorie mais on commence par tester avec plus bas
T = 4*60*60 #conversion en secondes
lamb = 1/300
M = int(1e5)

#On définit nu

a = [np.array([-1,1]),np.array([-3,-2,-1,1,2,3])]
pr = [np.array([0.5,0.5]),0.5*np.array([1./6.,1./3,1./2.,1./2.,1./3.,1./6.])]

#Choix de m=1 ou m=2
k=0

def liste_prix_sauts(lamb,T,k,P0):
    N = np.random.poisson(lamb*T) #le nombre de sauts suit une loi de Poisson de paramètre lambda*T
    liste_increments = np.random.choice(a[k],size=N,replace=True,p=pr[k])
    liste_prix = P0 + np.cumsum(liste_increments)

    return liste_prix


def MC_naif(M,lamb,T,k,P0): #k=0 si m=1, k=1 si m=2
    liste_bool_inf_0 = np.zeros(M) 

    for l in range(M):
        liste_prix = liste_prix_sauts(lamb,T,k,P0) #prix juste après les sauts
        if np.min(liste_prix) <0:
            liste_bool_inf_0[l] = 1

    proba_prix_negatif = np.mean(liste_bool_inf_0)
    return proba_prix_negatif

proba_prix_negatif = MC_naif(M,lamb,T,k,P0)
print(proba_prix_negatif)


0.02106
CPU times: user 5.12 s, sys: 1.01 ms, total: 5.12 s
Wall time: 5.13 s


In [0]:
###Version Peng-Wei naif avec astuce

from numba import jit

@jit(nopython=True)
def inf_echantillon(N, J, res, P0): # Function is compiled to machine code when called the first time
  for i in range(len(N) - 1):
    somme = P0
  
    for j in range(N[i], N[i + 1]):
      somme += J[j]
      if somme < 0:
        res[i] = True
        break
  return res

def trajectoire(P0, T, lamb, saut, size):
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
  J = saut(N[-1] + 1)
  res = np.zeros(size, dtype=bool)
  res = inf_echantillon(N, J, res, P0)
  proba = np.mean(res)
  return proba

In [0]:
%%time
print(trajectoire(P0, T, lamb, saut_1, size=int(1e7)))

#Donne 0.0214804 pour P0=15

0.0214391
CPU times: user 16.6 s, sys: 296 ms, total: 16.9 s
Wall time: 17 s


In [0]:
## Splitting et MCMC (Diane)

#On note Phi_T l'inf des prix entre 0 et T

def NiveauxSplitting(a,seuil,n,lamb,T,p,P0,k,J,pr):

    """
    Fonction qui renvoie une estimation des niveaux
    de splitting a_1, a_2, ..., a_k tels que P(Phi_T <= a_k | Phi_T <= a_{k-1}) = 0.1 = seuil
    Ces niveaux sont des quantiles d'une loi conditionnelle.
    On utilise l'inversion de la fonction de repartition empirique de 
    cette loi afin d'estimer un quantile par
    le quantile empirique.
    On a a = a_k < a_{k-1} < ... < a_0= + infini (dans notre problème, a = 0)
    La fonction renvoie quantiles = [a_1, ..., a_k]
    """
    ## Estimation du premier niveau a_1: c'est le 
    ## quantile d'une loi non conditionnelle.
    ## On l'estime ici par la methode ergodique

    liste_Phi = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,J,pr)

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,J,pr)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_Phi[l] = Phi(liste_sauts,P0)

    liste_Phi.sort()

    quantiles = np.array([liste_Phi[int(np.ceil(seuil*n))-1]])    

    while quantiles[-1] > a:
        print("Inside while")
        liste_Phi = np.zeros(n)

        
        while Phi(liste_sauts,P0)>=quantiles[-1]:
            liste_sauts = liste_sts(lamb,T,J,pr)
        ## Simulation du processus AR(1) conditionnel
    
        for l in range(n):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,J,pr)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)<quantiles[-1]:
                liste_sauts = new_liste_sauts

            liste_Phi[l] = Phi(liste_sauts,P0)
    
        liste_Phi.sort()
        quantiles = np.append(quantiles, liste_Phi[int(np.ceil(seuil*n))-1] )

    
    ## On selectionne les niveaux a_{k-1},..., a_1 strictement au dessus de a
    quantiles = quantiles[:-1]
    ## On rajoute a
    quantiles = np.append(quantiles,a)

    return quantiles


def Phi(liste_sauts,P0):#fonction qui renvoie l'inf des valeurs de X aux instants de saut
    #ie l'inf de P0+cumsum(incréments) 
    if len(liste_sauts[0]) == 0:
        return P0
    liste_prix = P0+np.cumsum(liste_sauts[1,:])
    prix_min = np.min(liste_prix)
    return prix_min

def liste_sts(lbda,T,J,pr):
    
    N = np.random.poisson(lbda*T)
    liste_temps_sauts = np.random.uniform(low = 0, high = T, size = N)
    liste_temps_sauts_triee = [np.sort(liste_temps_sauts)]
    liste_increments = [np.random.choice(J[k],size=N,replace=True,p=pr[k])]
    #renvoie un array de N colonnes et 2 lignes: 1ere ligne pour les temps des sauts (T_n), deuxième ligne pour leurs amplitudes (J_n)
    return np.concatenate((liste_temps_sauts_triee,liste_increments),axis=0)

def tri_temps(new_liste_sauts):
    ordre = [new_liste_sauts[0,:].argsort()]
    liste_sauts_triee = np.take_along_axis(new_liste_sauts, np.concatenate((ordre,ordre),axis=0), axis=1) 
    return liste_sauts_triee


def MCMC(M,p,lamb,liste_a,P0,k): 

    liste_pi = np.zeros(len(liste_a)) #estimateurs des probabilités conditionnelles

    liste_indicatrices = np.zeros(M,dtype=bool) 
    """
    le k-ieme élém. de liste_indicatrices vaut True si le prix devient négatif avant l'instant T lors du k-ieme essai; False sinon
    """

    #Loi non conditionnelle
    
    liste_sauts = liste_sts(lamb,T,J,pr) 


    for l in range(M):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,J,pr)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_indicatrices[l] = (Phi(liste_sauts,P0)<liste_a[0])

    liste_pi[0] = np.mean(liste_indicatrices)
    
        


    for k in range(1,len(liste_a)):
        liste_indicatrices = np.zeros(M,dtype=bool)  

        #Initialisation 

        while Phi(liste_sauts,P0)>=liste_a[k-1]:
          
            liste_sauts = liste_sts(lamb,T,J,pr)

      
        for l in range(M):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,J,pr)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)< liste_a[k-1]:
                liste_sauts = new_liste_sauts
            liste_indicatrices[l] = (Phi(liste_sauts,P0) < liste_a[k])


        liste_pi[k] = np.mean(liste_indicatrices)
    
    print(liste_pi)#debug

    proba_prix_negatif = np.prod(liste_pi)
        
    return proba_prix_negatif

In [0]:
P0 = 35 
T = 4*60*60 #conversion en secondes
lamb =  1/300 
n = int(1e4) 

#On définit nu

J = [np.array([-1,1]),np.array([-3,-2,-1,1,2,3])]
pr = [np.array([0.5,0.5]),0.5*np.array([1./6.,1./3,1./2.,1./2.,1./3.,1./6.])]
a=0

#Choix de m=1 ou m=2
k=1

#Choix des paramètres pour les niveaux de splitting et la simulation par chaîne de Markov
seuil = 0.05
p=0.5

liste_a = NiveauxSplitting(a,seuil,n,lamb,T,p,P0,k,J,pr)
print(liste_a)
#Donne [22. 14.  8.  3.  0.]

Inside while
[11.  0.]


In [0]:
M = int(1e5)
proba_prix_negatif = MCMC(M, p,lamb,liste_a,P0,k)
print(proba_prix_negatif)

#Réponse pour m=1 ie k=0 et P0=35 : proba = 3.333674930684664e-07

[0.04585 0.09552]
0.004379592


###  Q1 - 2 Quantile


In [0]:
#Peng-Wei naïf rapide

# Poisson composé naïf
def Poisson_compose(P0, T, lamb, saut, size):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = np.cumsum(N)                  # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)               # Sauts
  J = np.cumsum(J)
  M = J[N[1:]] - J[N[:-1]]          # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  return P0 + M # échantillon de taille (size)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question1_2_naive(size, quant, saut, P0, T, lamb):
  return quantile(Poisson_compose(P0, T, lamb, saut, size), size, quant)

In [0]:
%%time
size = int(1e6)
quant = 1e-4
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

#Donne 9 et 61 pour P0 = 35 et m=1 et niveau = 10^-4 
#Donne 5 et 65 pour P0 = 35 et m=1 et niveau = 10^-5 
#Donne 3 et 68 pour P0 = 35 et m=1 et niveau = 10^-6
#Donne -13 et 83 pour P0 = 35 et m=2 et niveau = 10^-4 
#Donne -21 et 92 pour P0 = 35 et m=2 et niveau = 10^-5
#Donne -25 et 94 pour P0 = 35 et m=2 et niveau = 10^-6

(-6, 46)
CPU times: user 1.41 s, sys: 4 ms, total: 1.41 s
Wall time: 1.42 s


In [0]:

#Diane MCMC/AR(1) (comme pour les calculs de NiveauxSplitting en Q1 -1)

def Quantile_ergodique(seuil,n,lamb,T,p,P0,k,J,pr): #renvoie le quantile de niveau "seuil" du processus de poisson composé

    liste_prix_final = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,J,pr) 

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,J,pr)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        liste_sauts = tri_temps(new_liste_sauts)

        liste_prix_final[l] = np.cumsum(liste_sauts[1,:])[-1] +P0

    liste_prix_final.sort()
  
    quantile = liste_prix_final[int(np.ceil(seuil*n))-1]    

  
    return quantile

In [0]:
#Diane naïf

def Quantile_naif(seuil,n,lamb,T,P0,k,J,pr):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N = np.random.poisson(lamb*T)
        increments = np.random.choice(J[k],size=N,replace=True,p=pr[k])
        
        liste_P_t[i] = np.sum(increments)+P0
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

In [0]:
P0 = 15
T = 4*60*60 #conversion en secondes
lamb =  1/300 
n = int(1e5) 

#On définit nu

J = [np.array([-1,1]),np.array([-3,-2,-1,1,2,3])]
pr = [np.array([0.5,0.5]),0.5*np.array([1./6.,1./3,1./2.,1./2.,1./3.,1./6.])]
a=0

#Choix de m=1 (k=0) ou m=2 (k=1)
k=1

#Choix des paramètres pour les niveaux de splitting et la simulation par chaîne de Markov
niveau = 1e-4
p=0.5

quantileGauche = Quantile_naif(niveau,n,lamb,T,P0,k,J,pr) #mettre Quantile_ergodique(niveau,n,lamb,T,p,P0,k,J,pr) ou Quantile_naif(niveau,n,lamb,T,P0,k,J,pr)
quantileDroit = Quantile_naif(1-niveau,n,lamb,T,P0,k,J,pr)
print("Quantile de niveau {} : {}".format(niveau,quantileGauche))
print("Quantile de niveau {} : {}".format(1-niveau,quantileDroit))

#Résultats pour Quantile_ergodique:
#Donne 9 et 61 pour P0 = 35 et k=0 et niveau = 10^-4 (c'est à peu près sym/P0 ce qui est rassurant)
#Donne 5 et 66 pour P0 = 35 et k=0 et niveau = 10^-5
#Donne 5 et 63 pour P0 = 35 et k=0 et niveau = 10^-6 (problème... faut augmenter n?) n : 10^5 -> 10^6 : on trouve 2 et 68 ; OK :D

#-15 et 83 pour P0=35 et k=1 et niveau = 10^-4 (sauts plus grands dc on va plus loin, logique; sym/0, logique)
#-16 et 90 pour P0=35 et k=1 et niveau = 10^-5
#Donne -25 et 97 pour P0 = 35 et k=1 et niveau = 10^-6 (en augmentant n de 10^5 à 10^6)

#Résultats Quantile_naïf (plus lent) (n=10^5):
#9 et 62
#5 et 66
#3 et 67
#-13 et 84
#-22 et 87
# -22 et 97(on passe à n=10^6)

Quantile de niveau 0.05 : -6.0
Quantile de niveau 0.95 : 36.0


In [0]:
#Calcul quantile par échantillonnage d'importance
#Voir slides amphi 2 p.12 pour calcul quantile avec changement de probabilité
#Ensuite voir amphi 3 pour changements de proba Poisson composés
#Différence importance sampling/échantillonage d'importance et changement de probabilité ?
#Transfo d'Esscher

###  Q2




In [0]:
#MC Naif Diane

lamb1 = 1/660
lamb2 = 1/110

def liste_sts2(lbda1,lbda2,T,J,pr):
    
    N1 = np.random.poisson(lbda1*T)
    N2 = np.random.poisson(lbda2*T)
    liste_temps_sauts1 = np.random.uniform(low = 0, high = T, size = N1)
    liste_temps_sauts2 = np.random.uniform(low = 0, high = T, size = N2)
    liste_temps_sauts = [np.concatenate(liste_temps_sauts1,liste_temps_sauts2)]
    liste_increments = [np.random.choice(J[k],size=N,replace=True,p=pr[k])]
    #renvoie un array de N colonnes et 2 lignes: 1ere ligne pour les temps des sauts (T_n), deuxième ligne pour leurs amplitudes (J_n)
    np.concatenate((liste_temps_sauts_triee,liste_increments),axis=0)
    return tri_temps()

In [148]:
#MC Naif PWC

def Q2_MC(P0, T, lamb1, lamb2, saut):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
    N1 = np.random.poisson(lam=lamb1 * T)
    T1 = np.sort(np.random.uniform(low=0.0, high=T, size=N1))
    J1 = saut(N1)

    N2 = np.random.poisson(lam=lamb2 * T)
    T2 = np.sort(np.random.uniform(low=0.0, high=T, size=N2))

    J2_0 = saut_1(1)[0]

    somme = P0
    index_T2 = 0
    for i in range(len(T1)):
        while index_T2 < N2 and T2[index_T2] < T1[i]:
            somme += J2_0
            J2_0 *= -1
            index_T2 += 1
        somme += J1[i]
    return somme                      # 1 échantillon

def Q2_MC_n(P0, T, lamb1, lamb2, saut, size):
    res = []
    for i in range(size):
        res.append(Q2_MC(P0, T, lamb1, lamb2, saut))
    return np.array(res)

In [7]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

saut_1 = lambda x: np.random.choice([-1, 1], size=x, p=[0.5, 0.5])
saut_2 = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6]))
P0 = 35
T = 4*60*60
lamb1 = 1/660
lamb2 = 1/110
saut = saut_1

In [149]:
Q2_MC_n(P0, T, lamb1, lamb2, saut_1, int(1e4))

array([32, 34, 33, ..., 29, 15, 31])

In [12]:
T1

array([  490.00205895,   761.00029958,  2397.68248461,  4121.06222689,
        4710.62229056,  4736.89308214,  4770.52004289,  5453.48773341,
        7890.36592896,  8080.23887538,  8253.25547813,  8325.95039769,
        9644.84182391, 10016.40780559, 11609.50886355, 11784.59066359,
       12930.21581953])

In [13]:
T2

array([1.38424231e+01, 3.53209736e+02, 4.08300918e+02, 7.02673238e+02,
       9.77753586e+02, 1.25364062e+03, 1.38202874e+03, 1.44900166e+03,
       1.67489464e+03, 1.91882459e+03, 1.94315634e+03, 2.03995328e+03,
       2.50915558e+03, 2.59799469e+03, 2.60505340e+03, 2.62719167e+03,
       2.69788007e+03, 3.23581758e+03, 3.39805019e+03, 3.44826411e+03,
       3.55314319e+03, 3.62070796e+03, 3.63399441e+03, 3.65257175e+03,
       3.66613002e+03, 3.69063062e+03, 3.89497022e+03, 4.00992427e+03,
       4.01105917e+03, 4.18006621e+03, 4.18075415e+03, 4.19546304e+03,
       4.53486134e+03, 5.03418692e+03, 5.04746705e+03, 5.07864662e+03,
       5.15539022e+03, 5.34470524e+03, 5.39521476e+03, 5.43773208e+03,
       5.78774684e+03, 5.86522510e+03, 6.13744312e+03, 6.22999100e+03,
       6.37182519e+03, 6.43423708e+03, 6.83434100e+03, 6.85522663e+03,
       6.90993552e+03, 6.91113006e+03, 6.97817180e+03, 7.05111049e+03,
       7.13726377e+03, 7.32918335e+03, 7.44924831e+03, 7.45813661e+03,
      